In [1]:
import cv2, numpy as np, time, threading, queue, traceback, sys, os, csv, json, subprocess
from collections import deque, defaultdict

# ---------- Configuration ----------
FPS_WINDOW = 1.0
DISPLAY_FPS = 70
VISUALIZE = True
DETECTION_SCALE = 0.5  # Downscale for ball detection speed

# Camera Configs (Device IDs and Settings)
CAM_CONFIGS = {
    "kreo1": {
        "dev": 4,
        "json": "../config/camera_tune_results/best_camera_settings_kreo1.json"
    },
    "kreo2": {
        "dev": 0,
        "json": "../config/camera_tune_results/best_camera_settings_kreo2.json"
    }
}

# HSV Config
HSV_CONFIG = {
    "kreo1": { "orange": {'hmin': 0, 'smin': 116, 'vmin': 160, 'hmax': 12, 'smax': 197, 'vmax': 255} },
    "kreo2": { "orange": {'hmin': 0, 'smin': 79, 'vmin': 181, 'hmax': 12, 'smax': 255, 'vmax': 255} }
}
DEFAULT_HSV = {'hmin': 0, 'smin': 100, 'vmin': 100, 'hmax': 25, 'smax': 255, 'vmax': 255}

# Detection Params
BASE_MIN_AREA = 100    
BASE_MAX_AREA = 20000  
CIRCULARITY_MIN = 0.5 
ASPECT_RATIO_MIN = 0.6
ASPECT_RATIO_MAX = 1.6   

# Logging
LOG_DIR = "../../data/direct_test_logs"
if not os.path.exists(LOG_DIR): os.makedirs(LOG_DIR)
LOG_FILENAME = f"{LOG_DIR}/log_{int(time.time())}.csv"

# AprilTag Config
DICT_TYPE = cv2.aruco.DICT_APRILTAG_36h11
def create_april_detector():
    aruco_dict = cv2.aruco.getPredefinedDictionary(DICT_TYPE)
    params = cv2.aruco.DetectorParameters()
    params.adaptiveThreshWinSizeMin = 3
    params.adaptiveThreshWinSizeMax = 35
    params.adaptiveThreshWinSizeStep = 2
    params.cornerRefinementMethod = cv2.aruco.CORNER_REFINE_SUBPIX
    params.useAruco3Detection = True # Uncomment if using OpenCV 4.7+ for speed boost
    params.cornerRefinementMaxIterations = 100
    params.cornerRefinementMinAccuracy = 0.01
    params.minMarkerPerimeterRate = 0.02
    params.maxMarkerPerimeterRate = 6.0
    params.polygonalApproxAccuracyRate = 0.02
    params.adaptiveThreshConstant = 7
    return cv2.aruco.ArucoDetector(aruco_dict, params)

# ---------- V4L2 Settings Application ----------
def apply_settings(json_path, dev_idx):
    try:
        if not os.path.exists(json_path):
            print(f"[WARN] Config not found: {json_path}. Skipping.")
            return
        with open(json_path, "r") as f:
            cfg = json.load(f)
        cmds = [
            ("exposure_time_absolute", cfg.get("exp")),
            ("gain", cfg.get("gain")),
            ("focus_absolute", cfg.get("focus")),
            ("brightness", cfg.get("brightness")),
        ]
        for ctrl, val in cmds:
            if val is None: continue
            subprocess.run(["v4l2-ctl", "-d", f"/dev/video{dev_idx}", "-c", f"{ctrl}={int(val)}"],
                           stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        print(f"[Settings] Applied to /dev/video{dev_idx}")
    except Exception as e:
        print(f"[Settings] Error applying {json_path}: {e}")

# ---------- Logging Worker ----------
log_queue = queue.Queue()

def logger_worker():
    try:
        with open(LOG_FILENAME, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow([
                "timestamp", "camera", 
                "ball_detected", "ball_x", "ball_y", "ball_area",
                "tag4_detected", "tag4_x", "tag4_y",
                "tag5_detected", "tag5_x", "tag5_y"
            ])
            while True:
                entry = log_queue.get()
                if entry is None: break
                writer.writerow(entry)
                log_queue.task_done()
    except Exception as e: print(f"[LOGGER ERROR] {e}")

log_thread = threading.Thread(target=logger_worker, daemon=True)
log_thread.start()

# ---------- Detection Helpers ----------
def get_orange_mask(bgr_img, hsv_dict):
    hsv = cv2.cvtColor(bgr_img, cv2.COLOR_BGR2HSV)
    lower = np.array([hsv_dict['hmin'], hsv_dict['smin'], hsv_dict['vmin']], dtype=np.uint8)
    upper = np.array([hsv_dict['hmax'], hsv_dict['smax'], hsv_dict['vmax']], dtype=np.uint8)
    mask = cv2.inRange(hsv, lower, upper)
    return cv2.GaussianBlur(mask, (5, 5), 0)

def find_ball_contours(mask, min_area, max_area):
    if mask is None: return []
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    candidates = []
    for c in contours:
        area = cv2.contourArea(c)
        if area < min_area or area > max_area: continue
        x,y,w,h = cv2.boundingRect(c)
        aspect = float(w)/h if h > 0 else 0
        if aspect > ASPECT_RATIO_MAX: continue
        perim = cv2.arcLength(c, True)
        if perim == 0: continue
        circularity = 4 * np.pi * area / (perim * perim)
        if circularity >= CIRCULARITY_MIN:
            candidates.append({"bbox": (x,y,w,h), "area": area, "centroid": (x+w//2, y+h//2)})
    candidates.sort(key=lambda d: d["area"], reverse=True)
    return candidates

# ---------- Processing Thread (One per Camera) ----------
class CameraProcessThread(threading.Thread):
    def __init__(self, name, config, detect_cache, lock):
        super().__init__(daemon=True)
        self.name = name
        self.dev_idx = config["dev"]
        self.json_path = config["json"]
        self.detect_cache = detect_cache
        self.lock = lock
        self.stop_flag = False
        
        # FPS Calculation
        self.fps_dq = deque()
        
        # Detection Setup
        self.min_area_scaled = BASE_MIN_AREA * (DETECTION_SCALE**2)
        self.max_area_scaled = BASE_MAX_AREA * (DETECTION_SCALE**2)
        self.hsv_vals = HSV_CONFIG.get(name, {}).get("orange", DEFAULT_HSV)
        self.aruco_detector = create_april_detector()

        # Apply Hardware Settings
        apply_settings(self.json_path, self.dev_idx)

        # Initialize Camera
        self.cap = cv2.VideoCapture(self.dev_idx, cv2.CAP_V4L2)
        self.cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter_fourcc(*"MJPG"))
        self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
        self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
        self.cap.set(cv2.CAP_PROP_FPS, 60)
        self.cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)

        if not self.cap.isOpened():
            print(f"[ERROR] Could not open {self.name} on /dev/video{self.dev_idx}")
            self.stop_flag = True

    def update_fps(self, ts):
        self.fps_dq.append(ts)
        while self.fps_dq and (ts - self.fps_dq[0]) > FPS_WINDOW:
            self.fps_dq.popleft()
        return len(self.fps_dq) / FPS_WINDOW

    def run(self):
        print(f"[{self.name}] Thread started. Capturing & Detecting...")
        while not self.stop_flag:
            ret, frame = self.cap.read()
            if not ret:
                time.sleep(0.001)
                continue
            
            ts = time.time()
            fps = self.update_fps(ts)

            # --- DETECTION LOGIC ---
            
            # 1. Ball (Scaled)
            if DETECTION_SCALE != 1.0:
                frame_small = cv2.resize(frame, None, fx=DETECTION_SCALE, fy=DETECTION_SCALE, interpolation=cv2.INTER_NEAREST)
            else:
                frame_small = frame
            
            mask = get_orange_mask(frame_small, self.hsv_vals)
            balls = find_ball_contours(mask, self.min_area_scaled, self.max_area_scaled)

            ball_data = {"detected": False, "x": "", "y": "", "area": ""}
            best_ball = None 

            if balls:
                b = balls[0]
                scale_inv = 1.0 / DETECTION_SCALE
                bx, by, bw, bh = b["bbox"]
                real_x = int(bx * scale_inv)
                real_y = int(by * scale_inv)
                real_w = int(bw * scale_inv)
                real_h = int(bh * scale_inv)
                real_area = int(b["area"] * (scale_inv**2))
                cx, cy = real_x + real_w//2, real_y + real_h//2
                
                ball_data = {"detected": True, "x": cx, "y": cy, "area": real_area}
                best_ball = {"bbox": (real_x, real_y, real_w, real_h), "centroid": (cx, cy)}

            # 2. AprilTag (Full Res)
            gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            corners, ids, _ = self.aruco_detector.detectMarkers(gray)
            
            tag4_data = {"detected": False, "x": "", "y": ""}
            tag5_data = {"detected": False, "x": "", "y": ""}
            found_tags_viz = []

            if ids is not None:
                ids_flat = ids.flatten()
                for i, tag_id in enumerate(ids_flat):
                    c = corners[i][0]
                    cx = int(np.mean(c[:, 0]))
                    cy = int(np.mean(c[:, 1]))
                    found_tags_viz.append({"id": tag_id, "corners": corners[i]})
                    if tag_id == 4:
                        tag4_data = {"detected": True, "x": cx, "y": cy}
                    elif tag_id == 5:
                        tag5_data = {"detected": True, "x": cx, "y": cy}

            # 3. Log
            log_queue.put([
                f"{ts:.3f}", self.name,
                ball_data["detected"], ball_data["x"], ball_data["y"], ball_data["area"],
                tag4_data["detected"], tag4_data["x"], tag4_data["y"],
                tag5_data["detected"], tag5_data["x"], tag5_data["y"]
            ])

            # 4. Update Cache (For Viz)
            if VISUALIZE:
                with self.lock:
                    self.detect_cache[self.name] = {
                        "img": frame, # Copy not needed unless viz modifies it, but safer if thread writes to it
                        "ball": best_ball,
                        "tags": found_tags_viz,
                        "fps": fps
                    }

        self.cap.release()
        print(f"[{self.name}] Stopped.")

    def stop(self):
        self.stop_flag = True

# ---------- Main Loop ----------
detect_cache = {}
detect_lock = threading.Lock()
threads = []

print("[Main] Starting threads...")
for name, cfg in CAM_CONFIGS.items():
    t = CameraProcessThread(name, cfg, detect_cache, detect_lock)
    t.start()
    threads.append(t)

print("[Main] Running. Press ESC to exit.")
last_show = time.time()

try:
    while True:
        # Throttled Visualization
        if VISUALIZE and (time.time() - last_show) > (1.0/DISPLAY_FPS):
            with detect_lock:
                has_data = all(name in detect_cache for name in CAM_CONFIGS)
                if has_data:
                    imgs = []
                    for name in ["kreo1", "kreo2"]:
                        data = detect_cache[name]
                        im = data["img"].copy()
                        fps = data["fps"]
                        
                        # Draw FPS
                        cv2.putText(im, f"{name} FPS: {fps:.1f}", (10,30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)
                        
                        # Draw Ball
                        if data["ball"]:
                            bx,by,bw,bh = data["ball"]["bbox"]
                            cx,cy = data["ball"]["centroid"]
                            cv2.rectangle(im, (bx,by), (bx+bw, by+bh), (0,165,255), 2)
                            cv2.circle(im, (cx,cy), 5, (0,0,255), -1)
                        
                        # Draw Tags
                        for tag in data["tags"]:
                            cv2.aruco.drawDetectedMarkers(im, [tag["corners"]], np.array([[tag["id"]]]))
                        
                        imgs.append(im)
                    
                    # Stack
                    l_im, r_im = imgs[0], imgs[1]
                    h = min(l_im.shape[0], r_im.shape[0])
                    if l_im.shape[0] != h: l_im = cv2.resize(l_im, (int(l_im.shape[1]*h/l_im.shape[0]), h))
                    if r_im.shape[0] != h: r_im = cv2.resize(r_im, (int(r_im.shape[1]*h/r_im.shape[0]), h))
                    
                    tile = np.hstack([l_im, r_im])
                    cv2.imshow("Direct Test", tile)
                    last_show = time.time()

        elif not VISUALIZE:
            # Headless Status
            with detect_lock:
                status = " | ".join([f"{k}: {v.get('fps',0):.1f} FPS" for k,v in detect_cache.items()])
                sys.stdout.write(f"\r{status}")
                sys.stdout.flush()
            time.sleep(0.1)

        if cv2.waitKey(1) & 0xFF == 27:
            break

except KeyboardInterrupt:
    pass
finally:
    print("\nStopping threads...")
    for t in threads: t.stop()
    for t in threads: t.join()
    log_queue.put(None)
    log_thread.join()
    cv2.destroyAllWindows()
    print("Done.")

[Main] Starting threads...
[Settings] Applied to /dev/video4
[kreo1] Thread started. Capturing & Detecting...
[Settings] Applied to /dev/video0
[kreo2] Thread started. Capturing & Detecting...
[Main] Running. Press ESC to exit.

Stopping threads...
[kreo1] Stopped.
[kreo2] Stopped.
Done.
